---

Here use your CSV from WIX: Example! "Pole Studios (2).csv", replace the Name of the CSV inside the "", Run All Cells!

In [15]:
import pandas as pd

# Load initial URLs
hop_dataframe = pd.read_csv("Pole Studios.csv")
hop_dataframe.head(3)

,Quelle,Pole Studio,PLZ,Stadt,Land,Adresse,Telefonnummer,E-Mail-Adresse,Webseite,Facebook,...,ID,Owner,Created Date,Updated Date,Bildergalerie,Richtigkeit der Angaben,Pole Studios (Pole Studio),Online Studios (Pole Studio),Inaktiv,Löschvermerk
0,https://www.eversports.de/sw/seemannsbraut-pol...,Seemannsbraut Poledance Flensburg,24941,Flensburg,Deutschland,"{""subdivisions"":[{""code"":""SH"",""name"":""Schleswi...",15163300769,seemannsbraut-poledance@web.de,https://www.seemannsbraut-poledance.de/,https://www.facebook.com/SeemannsbrautPoledance,...,01df35bd-209a-4520-a7bc-d109b9aaf2f6,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/seemannsbraut-poledance-flensburg,/onlinestudio/seemannsbraut-poledance-flensburg,NaN,NaN
1,https://www.eversports.de/sw/tsg-blau-gold-gie...,TSG Blau Gold Ballett- und Pole-dance- Studio,35390,Gießen,Deutschland,"{""subdivisions"":[{""code"":""HE"",""name"":""Hessen"",...",0491 621606468,tanz-giessen@gmx.de,https://www.tsg-giessen.de/angebote/tanz/poled...,https://www.facebook.com/poledancegiessen.de,...,0233997c-a277-4179-990a-347e1cea347e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:49Z,NaN,NaN,/polestudio/tsg-blau-gold-ballett--und-pole-da...,/onlinestudio/tsg-blau-gold-ballett--und-pole-...,NaN,NaN
2,NaN,Pole Stars Osnabrück,49080,Osnabrück,Deutschland,"{""subdivisions"":[{""code"":""NDS"",""name"":""Nieders...",01573 1810262,info@polestars-osnabrueck.de,http://www.polestars-osnabrueck.de/,https://www.facebook.com/PoleStarsOsnabrueck.de/,...,04673b9d-d8b2-4caf-8632-d349eab2cd4e,284241e1-0e53-4183-b67f-34edcfb7c8b1,2022-11-18T11:57:31Z,2024-02-04T18:00:48Z,NaN,NaN,/polestudio/pole-stars-osnabr%C3%BCck,/onlinestudio/pole-stars-osnabr%C3%BCck,NaN,NaN


---

Url Reconstruction

In [16]:
hop_polestudios = hop_dataframe["Pole Studios (Pole Studio)"].to_list()

initial_urls = hop_polestudios

base_url = "https://www.eversports.de/s"

#initial_urls = list(set(hop_dataframe["Pole Studios (Pole Studio)"]))
len(initial_urls)

330

In [17]:
url = []
for url_part in initial_urls:
    url.append(base_url+url_part[11:])
url[:10]

['https://www.eversports.de/s/seemannsbraut-poledance-flensburg',
 'https://www.eversports.de/s/tsg-blau-gold-ballett--und-pole-dance--studio',
 'https://www.eversports.de/s/pole-stars-osnabr%C3%BCck',
 'https://www.eversports.de/s/aerial-art-studio',
 'https://www.eversports.de/s/k%C3%B6rpernah',
 'https://www.eversports.de/s/dance-moves-wolfenb%C3%BCttel',
 'https://www.eversports.de/s/mypole',
 'https://www.eversports.de/s/tanzstudio-michel',
 'https://www.eversports.de/s/pole-sensation',
 'https://www.eversports.de/s/poledance-rostock']

In [18]:
initial_urls = url

---

Scraper Code

In [19]:
import pandas as pd
from a_PyCaller import process_urls
from tqdm import tqdm
from datetime import datetime

def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data):
    data = process_urls([url])

    if data:
        for key, df in data.items():
            if df is not None and not df.empty:
                tqdm.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - INFO - Scraping Data from: {url}")
                tqdm.write(f"{key.replace('_', ' ').title()}: {len(df)} entries")

                # Update the appropriate DataFrame
                if key == 'pole_studio_data':
                    all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
                elif key == 'workshops_data':
                    all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
                elif key == 'workshop_details':
                    all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

    # Add URLs to DataFrame
    all_urls_data = pd.concat([all_urls_data, pd.DataFrame({'URL': [url]})], ignore_index=True)

    return all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data

def main():
    # # Load initial URLs
    # all_found_urls_s = pd.read_csv("your_output_file.csv")
    # initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

    # Initialize DataFrames
    all_pole_studio_data = pd.DataFrame()
    all_workshops_data = pd.DataFrame()
    all_workshop_details_data = pd.DataFrame()
    all_urls_data = pd.DataFrame(columns=['URL'])

    # Process each URL with tqdm
    with tqdm(initial_urls, desc="Processing URLs", dynamic_ncols=True) as pbar:
        for url in pbar:
            all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data = process_and_print_results(
                url, all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data
            )

    # Export DataFrames to CSV files
    # all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
    # all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
    # all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)
    # all_urls_data.to_csv("All_URLs.csv", index=False)

    # Return the final DataFrames
    return all_pole_studio_data, all_workshops_data, all_workshop_details_data, all_urls_data

# Run the main function and get the final DataFrames
result_pole_studio, result_workshops, result_workshop_details, result_urls = main()


Processing URLs:   0%|          | 0/330 [00:00<?, ?it/s]

Starting URL reconstruction...


Processing URLs:   0%|          | 1/330 [00:01<10:04,  1.84s/it]

Error during HTTP request: 404 Client Error: Not Found for url: https://www.eversports.de/s/seemannsbraut-poledance-flensburg
Converting to DataFrame...
Validating URLs...

Processing completed.
-------------------------------- 

Starting URL reconstruction...


Processing URLs:   1%|          | 2/330 [00:03<10:05,  1.85s/it]

...

Converting to DataFrame...
Validating URLs...


2024-02-21 14:01:05,155 - INFO - URL https://www.eversports.de/s/one8dance-studio is valid.


Scraping Pole Studio Data from https://www.eversports.de/s/one8dance-studio...


Processing URLs: 100%|██████████| 330/330 [19:33<00:00,  3.56s/it]


Processing completed.
-------------------------------- 

2024-02-21 14:01:05 - INFO - Scraping Data from: https://www.eversports.de/s/one8dance-studio
Pole Studio Data: 1 entries


In [20]:
result_pole_studio

,PoleStudio_Name,Adresse,PLZ,Stadt,Straße,Buttons,Pole Studio Beschreibung,E-Mail,Homepage,Telefon,URL_S,Art,Angebot,Created Date,Updated Date
0,Tanzstudio Michel,"[Schmalweg 50 , 55252 Wiesbaden]",55252,Wiesbaden,Schmalweg 50,"[Übersicht, Klassen, Preise, Team]",Das etablierte Tanzstudio in Wiesbaden bietet ...,t.michel.9@gmx.de,https://www.tanzstudio-michel.de/,+49 15115125871,https://www.eversports.de/s/tanzstudio-michel,"[1 Rezensionen, Fitness, Tanzen, Poledance, St...",Zum Stundenplan,2024-02-21 13:41:54,2024-02-21 13:41:54
1,Polestation Academy,"[Marktstraße 20b , 6971 Hard]",6971,Hard,Marktstraße 20b,"[Übersicht, Klassen, Videos, Preise, Team]",Polestation Academy ist ein Ort für Pole und A...,blum.n@gmx.at,http://www.polestation.com,+43 6642006568,https://www.eversports.de/s/polestation-academy,"[Poledance, Poledance, Poledance, Poledance, P...",Zum Stundenplan,2024-02-21 13:42:03,2024-02-21 13:42:03
2,Poledance 4 you,"[Naugarder Straße 14, 10409 Berlin]",10409,Berlin,Naugarder Straße 14,[Übersicht],Das Motto des Berliner Pole Dance Studios laut...,info@poledance4you.de,https://www.poledance4you.de/,None,https://www.eversports.de/s/poledance-4-you,"[(Noch keine Bewertungen), Poledance]",None,2024-02-21 13:42:10,2024-02-21 13:42:10
3,Luftbonbon,"[Königstraße 45 , 32547 Bad Oeynhausen]",32547,Bad,Königstraße 45,"[Übersicht, Klassen, Workshops, Preise, Team]",Entdecke mit uns die vielseitige Welt der Luft...,info@luftbonbon.com,https://www.luftbonbon.com/,01622572081,https://www.eversports.de/s/luftbonbon,"[Poledance, Aerial Hoop, Poledance, Stärkung/F...",Zum Stundenplan,2024-02-21 13:42:14,2024-02-21 13:42:14
4,Pole Dance Beauties,"[Edmund-von-Lippmann-Straße 9 , 06112 Halle (...",06112,Halle,Edmund-von-Lippmann-Straße 9,"[Übersicht, Klassen, Trainings, Videos, Preise...",Pole Dance Beauties - Dein Pole Dance Studio i...,kontakt@poledancebeauties.de,http://www.poledancebeauties.de,+49 1797553246,https://www.eversports.de/s/pole-dance-beauties,"[Aerial Hoop, Pilates, Krafttraining, Stretchi...",1 Angebot für Neukund:innen,2024-02-21 13:42:27,2024-02-21 13:42:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,The Basement - Pole Dance and Movement,"[Christian-Pleß-Straße 11-13, 63069 Offenbach]",63069,Offenbach,Christian-Pleß-Straße 11-13,[Übersicht],Das Studio The Basement - Pole Dance and Movem...,info@the-basement.eu,http://the-basement.eu/,None,https://www.eversports.de/s/the-basement-pole-...,"[36 Rezensionen, Tanzen, Poledance, Online Kla...",1 Angebot für Neukund:innen,2024-02-21 14:00:28,2024-02-21 14:00:28
99,Aerial Studio,"[Mississippidreef 2 , 3565 CG Utrecht]",3565,CG,Mississippidreef 2,"[Übersicht, Klassen, Videos, Preise, Team]",Diese Sportanlage bietet eine Vielzahl an Spor...,info@aerial-studio.nl,https://www.aerial-studio.nl,+31 648539599,https://www.eversports.de/s/aerial-studio,"[Poledance, Aerial Yoga, Aerial Dance, Aerial ...",2 Angebote für Neukund:innen,2024-02-21 14:00:49,2024-02-21 14:00:49
100,Delicate Pole,"[Am Gorzberg 12 , 17489 Greifswald]",17489,Greifswald,Am Gorzberg 12,"[Übersicht, Klassen, Trainings, Preise, Team]",Im schönen Greifswald bieten wir in unserem St...,delicate_pole@gmx.de,None,+49 1724437767,https://www.eversports.de/s/delicate-pole,"[18 Rezensionen, Tanzen, Poledance, Akrobatik,...",Zum Stundenplan,2024-02-21 14:00:56,2024-02-21 14:00:56
101,PoleFitness Leipzig,"[Salomonstraße 18B , 04103 Leipzig]",04103,Leipzig,Salomonstraße 18B,"[Übersicht, Klassen, Preise, Team]",Du suchst ein ansprechendes Krafttraining? Du ...,info@polefitness-leipzig.de,https://polefitness-leipzig.de/,+49 1629269299,https://www.eversports.de/s/polefitness-leipzig,"[Fitness, Poledance, Poledance, Stretching, St...",Zum Stundenplan,2024-02-21 14:00:59,2024-02-21 14:00:59


In [21]:
result_workshops

,Workshop Name,Workshop Date,Workshop Price,Workshop Units,Workshop Studio,Workshop Address,URL_E
0,Spagat,Feb05bis Feb 26,Ausgebucht,4 Einheiten,Luftbonbon,"Königstraße 45, 32547 Bad Oeynhausen",https://www.eversports.de/e/workshop/60kU5cU
1,Kopfstand Technik Online Workshop,Feb24Samstag,"ab 44,00 €",1 Einheit,Yoga and Pole Art by Selina,"Online *, World Wide",https://www.eversports.de/e/workshop/p_eKWTW
2,Atemtechniken Online Workshop,März16Samstag,"ab 44,00 €",1 Einheit,Yoga and Pole Art by Selina,"Online *, World Wide",https://www.eversports.de/e/workshop/1PmB24I
3,Split & Straddle Technik Online Workshop,März22Freitag,"ab 44,00 €",1 Einheit,Yoga and Pole Art by Selina,"Online *, World Wide",https://www.eversports.de/e/workshop/Zk4neM3
4,Split on Pole (Intermediate) Online Workshop,März29Freitag,"ab 44,00 €",1 Einheit,Yoga and Pole Art by Selina,"Online *, World Wide",https://www.eversports.de/e/workshop/VnmCBGD
...,...,...,...,...,...,...,...
135,Rock the Floor mit Emmy,Apr07Sonntag,Ausgebucht,1 Einheit,Da Becca Flow and Fit,"Eickeler Markt 3A, 44651 Herne",https://www.eversports.de/e/workshop/NrFmto0
136,Get your Shoulderstand - Floorworktechnik mit ...,Apr28Sonntag,"ab 25,00 €",1 Einheit,Da Becca Flow and Fit,"Eickeler Markt 3A, 44651 Herne",https://www.eversports.de/e/workshop/km4K7Yv
137,"""Fummeln für Fortgeschrittene"" Pole choreo mit...",Mai26Sonntag,Ausgebucht,1 Einheit,Da Becca Flow and Fit,"Eickeler Markt 3A, 44651 Herne",https://www.eversports.de/e/workshop/B2Sz3mv
138,"""Fummeln für Fortgeschrittene"" Chairdance Cho...",Mai26Sonntag,"ab 34,00 €",1 Einheit,Da Becca Flow and Fit,"Eickeler Markt 3A, 44651 Herne",https://www.eversports.de/e/workshop/HhyMbSA


In [22]:
result_workshop_details

,Workshop Name,Description,Studio Name,Location,Art,Date,URL_E,Time
0,Spagat,Straddle & Middlesplit stehen ganz oben auf De...,Luftbonbon,20:20 - 21:20,Stärkung/Flexibilität,"Begann am Montag, 5. Februar 2024",https://www.eversports.de/e/workshop/60kU5cU,"Königstraße 45, 32547 Bad Oeynhausen"
1,Kopfstand Technik Online Workshop,Der Kopfstand ist die erste Basis für viel wei...,Yoga and Pole Art by Selina,Diese Aktivität wird im Online Stream übertrag...,Vinyasa Yoga,"Samstag, 24. Februar 2024",https://www.eversports.de/e/workshop/p_eKWTW,18:00 - 19:30
2,Atemtechniken Online Workshop,Inhalte des Workshops: Aufbau des Workshops: ...,Yoga and Pole Art by Selina,Diese Aktivität wird im Online Stream übertrag...,Meditation,"Samstag, 16. März 2024",https://www.eversports.de/e/workshop/1PmB24I,18:00 - 19:30
3,Split & Straddle Technik Online Workshop,Du trainierst schon länger an deinem Spagat od...,Yoga and Pole Art by Selina,Diese Aktivität wird im Online Stream übertrag...,Vinyasa Yoga,"Freitag, 22. März 2024",https://www.eversports.de/e/workshop/Zk4neM3,18:00 - 19:30
4,Split on Pole (Intermediate) Online Workshop,Du liebst Spagat Figuren und brauchst neue Var...,Yoga and Pole Art by Selina,Diese Aktivität wird im Online Stream übertrag...,Poledance,"Freitag, 29. März 2024",https://www.eversports.de/e/workshop/VnmCBGD,12:00 - 13:30
...,...,...,...,...,...,...,...,...
134,Rock the Floor mit Emmy,Emmy bringt uns eine ihrer coolen Rockigen cho...,Da Becca Flow and Fit,Einzelticket,Floorwork,"Sonntag, 7. April 2024",https://www.eversports.de/e/workshop/NrFmto0,16:00 - 17:30
135,Get your Shoulderstand - Floorworktechnik mit ...,"Balance, Kraft; und ein ganz neues Körpergefüh...",Da Becca Flow and Fit,Einzelticket,Floorwork,"Sonntag, 28. April 2024",https://www.eversports.de/e/workshop/km4K7Yv,13:30 - 15:00
136,"""Fummeln für Fortgeschrittene"" Pole choreo mit...",Flirtmodus on! Du hast Lust darauf dich sexy a...,Da Becca Flow and Fit,Einzelticket,Poledance,"Sonntag, 26. Mai 2024",https://www.eversports.de/e/workshop/B2Sz3mv,12:00 - 13:30
137,"""Fummeln für Fortgeschrittene"" Chairdance Cho...",Weil sitzen auch ultra sexy sein kann. Jannah ...,Da Becca Flow and Fit,Einzelticket,Chairdance,"Sonntag, 26. Mai 2024",https://www.eversports.de/e/workshop/HhyMbSA,14:00 - 15:30


In [23]:
result_urls

,URL
0,https://www.eversports.de/s/seemannsbraut-pole...
1,https://www.eversports.de/s/tsg-blau-gold-ball...
2,https://www.eversports.de/s/pole-stars-osnabr%...
3,https://www.eversports.de/s/aerial-art-studio
4,https://www.eversports.de/s/k%C3%B6rpernah
...,...
325,https://www.eversports.de/s/delicate-pole
326,https://www.eversports.de/s/polefitness-leipzig
327,https://www.eversports.de/s/souldance
328,https://www.eversports.de/s/pole-heroes-pole-%...


---

Test V7

In [24]:
result_pole_studio, result_workshops, result_workshop_details, result_urls

(                            PoleStudio_Name  \
 0                         Tanzstudio Michel   
 1                       Polestation Academy   
 2                           Poledance 4 you   
 3                                Luftbonbon   
 4                       Pole Dance Beauties   
 ..                                      ...   
 98   The Basement - Pole Dance and Movement   
 99                            Aerial Studio   
 100                           Delicate Pole   
 101                     PoleFitness Leipzig   
 102                        One8dance Studio   
 
                                                Adresse    PLZ       Stadt  \
 0                    [Schmalweg 50 ,  55252 Wiesbaden]  55252   Wiesbaden   
 1                       [Marktstraße 20b ,  6971 Hard]   6971        Hard   
 2                 [Naugarder Straße 14,  10409 Berlin]  10409      Berlin   
 3             [Königstraße 45 ,  32547 Bad Oeynhausen]  32547         Bad   
 4    [Edmund-von-Lippmann-Straß

In [25]:
# import pandas as pd
# from a_PyCaller import process_urls
# from tqdm import tqdm
# from datetime import datetime

# def process_and_print_results(url, all_pole_studio_data, all_workshops_data, all_workshop_details_data):
#     data = process_urls([url])

#     if data:
#         for key, df in data.items():
#             if df is not None and not df.empty:
#                 tqdm.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - INFO - Scraping Data from: {url}")
#                 tqdm.write(f"{key.replace('_', ' ').title()}: {len(df)} entries")

#                 # Update the appropriate DataFrame
#                 if key == 'pole_studio_data':
#                     all_pole_studio_data = pd.concat([all_pole_studio_data, df], ignore_index=True)
#                 elif key == 'workshops_data':
#                     all_workshops_data = pd.concat([all_workshops_data, df], ignore_index=True)
#                 elif key == 'workshop_details':
#                     all_workshop_details_data = pd.concat([all_workshop_details_data, df], ignore_index=True)

#     return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# def main():
#     # # Load initial URLs
#     # all_found_urls_s = pd.read_csv("your_output_file.csv")
#     # initial_urls = list(set(all_found_urls_s["0"])) # Delete "[:3]" to scrape all Urls

#     # Initialize DataFrames
#     all_pole_studio_data = pd.DataFrame()
#     all_workshops_data = pd.DataFrame()
#     all_workshop_details_data = pd.DataFrame()

#     # Process each URL with tqdm
#     with tqdm(initial_urls, desc="Processing URLs", dynamic_ncols=True) as pbar:
#         for url in pbar:
#             all_pole_studio_data, all_workshops_data, all_workshop_details_data = process_and_print_results(
#                 url, all_pole_studio_data, all_workshops_data, all_workshop_details_data
#             )

#     # # Export DataFrames to CSV files
#     all_pole_studio_data.to_csv("Pole_Studio_Übersicht_S.csv", index=False)
#     all_workshops_data.to_csv("Workshop_Liste_SW.csv", index=False)
#     all_workshop_details_data.to_csv("Workshop_Übersicht_E.csv", index=False)

#     # Return the final DataFrames
#     return all_pole_studio_data, all_workshops_data, all_workshop_details_data

# # Run the main function and get the final DataFrames
# result_pole_studio, result_workshops, result_workshop_details = main()


In [26]:
# result_pole_studio

---

In [27]:
# result_workshops

In [28]:
# result_workshop_details